In [3]:
import numpy as np 
import math 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

An naive implementation of AdaBoostClassifier. 

## AdaBoost Pseudocode
Assign samples $i$ the weight of $d_{1i} = 1 / N$ (equal weights, $N$ is the number of samples).

FOR weak learner t in 1..T

- Train weak learner using data wegithed by $d_{ti}$, produces a classfier $h_t$
- Calculate classification error $$\epsilon_t = \frac{\sum_i|y_i != pred_i|}{N}$$
- Calculate coefficient alpha $$\alpha_t = 0.5 \text{ln}(\frac{1 - \epsilon_t}{\epsilon_t})$$
- Update sample weights $$\omega = \frac{\omega \cdot \exp(-\alpha_t \cdot y \cdot h_t(X))}{Z_t}$$, where $h(X)$ is prediction of $t$, and $Z_t = sum(\omega)$ is a normalization factor.
    Or more specifically, for sample $i$, 
$$\omega_{t+1, i} = \frac{\omega_{t, i} \cdot \exp(-\alpha_t \cdot y_i \cdot h_t(x_i))}{Z_t}$$, and $y_i \cdot h_t(x_i)$ is 1 if prediction is right, and -1 otherwise. 

ENDFOR

The final classifier is: $$H(x) = \text{sign} (\sum_{t=1}^T \alpha_t \cdot h_t(X))$$

In [1]:
import numpy as np 

class DecisionStump:
    def __init_(self):
        self.polarity = 1
        self.feature_idx = None
        self.threshold = None
        self.weight = None # estimator weight, not sample weight

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_idx]
        y_preds = np.ones(n_samples)
        if self.polarity == 1:
            y_preds[X_column <= self.threshold] = -1
        else:
            y_preds[X_column > self.threshold] = -1
        return y_preds

    
class AdaBoost:
    def __init__(self, n_estimators=5):
        self.n_estimators = n_estimators
    
    def _boost(self, X, y, sample_weight):
        '''Implement a single boost using the SAMME discrete algorithm.'''
        estimator = DecisionStump()
        min_error = 0x3f3f3f3f
        n_samples, n_features = X.shape

        for idx in range(n_features):
            X_column = X[:, idx]
            for t in np.unique(X_column):
                preds = np.ones(n_samples)
                preds[X_column <= t] = -1

                '''Use np.sum instead of sum speeds up calculation and improve perfromance'''
                error = np.sum(sample_weight[preds != y])

                polarity = 1 if error <= 0.5 else -1
                error = min(error, 1 - error)

                if error < min_error:
                    min_error = error
                    estimator.polarity = polarity
                    estimator.threshold = t 
                    estimator.feature_idx = idx 

        estimator.weight = 0.5 * np.log((1 - error) / (error + 1e-10))
        y_preds = estimator.predict(X)
        sample_weight *= np.exp(- estimator.weight * y * y_preds)
        sample_weight /= np.sum(sample_weight)

        self.estimators.append(estimator)
        return sample_weight
        
        
    def fit(self, X, y):
        '''Build a boosted classifier from the training set (X, y)'''
        n_samples, n_features = X.shape
        sample_weight = np.full(n_samples, 1 / n_samples)
        self.estimators = []
        
        for _ in range(self.n_estimators): # update sample_weight at each round
            sample_weight = self._boost(X, y, sample_weight)

        return self
    
    def predict(self, X):
        y_preds = np.sum([c.weight * c.predict(X) for c in self.estimators], axis=0)
        sign_preds = np.sign(y_preds)
        return sign_preds
    
np.full(5, 1)        

array([1, 1, 1, 1, 1])

In [6]:
from sklearn import datasets
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = datasets.load_breast_cancer()
X, y = data.data, data.target 
y[y == 0] = -1
n_estimators=10

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# =====================================================================
'''Our Naive Adaboost
'''
model = AdaBoost(n_estimators)
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("Naive AdaBoost Accuracy score", accuracy_score(y_val, y_preds))
# print(model.clfs)

# =====================================================================
'''Compore to AdaBoostClassifier using DecisionTreeClassifier as its decision stump
'''
dt_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
dt_stump.fit(X_train, y_train)
ada_dtc = AdaBoostClassifier(
    base_estimator=dt_stump,
    learning_rate=1,
    n_estimators=n_estimators,
    algorithm="SAMME.R")
ada_dtc.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("AdaBoost DecisionTree Accuracy score", accuracy_score(y_val, y_preds))


# =====================================================================
'''Compore to AdaBoostClassifier using XGBClassifier as its decision stump
'''
xgb_stump = xgboost.XGBClassifier()
xgb_stump.fit(X_train, y_train)
ada_real = AdaBoostClassifier(
    base_estimator=xgb_stump,
    learning_rate=1,
    n_estimators=10,
    algorithm="SAMME.R")
ada_real.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("AdaBoost XGB Accuracy score", accuracy_score(y_val, y_preds))

# =====================================================================
''' Compare to xgboost classifier
'''
import xgboost
# model = xgboost.XGBClassifier()
model = AdaBoostClassifier()
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("XGBoost Accuracy score", accuracy_score(y_val, y_preds))


# =====================================================================
''' Compare to Decision Tree
'''
dc = DecisionTreeClassifier(max_depth=10, random_state=0)
dc.fit(X_train, y_train)
y_preds = dc.predict(X_val)
print("Decision Tree Accuracy score", accuracy_score(y_val, y_preds))

Accuracy score 0.9649122807017544
AdaBoost DecisionTree Accuracy score 0.9649122807017544
AdaBoost XGB Accuracy score 0.9649122807017544
XGBoost Accuracy score 0.956140350877193
Decision Tree Accuracy score 0.9122807017543859
